In [1]:
# Install the required modules
!pip install transformers
!pip install pandas
!pip install numpy
!pip install tqdm
!pip install sklearn

from google.colab import drive
drive.mount('/content/gdrive')

# !nvidia-smi

     |████████████████████████████████| 317kB 5.1MB/s 
     |████████████████████████████████| 645kB 54.5MB/s 
     |████████████████████████████████| 860kB 63.2MB/s 
     |████████████████████████████████| 1.0MB 49.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=c598ebb1c1121fd35ed65ebc480e9919273670a8af93c106f6ff2aedabe9967c
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your 

In [0]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import *

In [0]:
# Embeddings can be derived from the last 1 or 4 layers, to reduce the computational cost, we used only the last layer.

class Embeddings:
    LAST_LAYER = 1
    LAST_4_LAYERS = 2
    def __init__(self):
        self._tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self._bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self._bert_model.eval()

    def tokenize(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: tokenized sentence based on word piece model ['List']
        """
        marked_sentence = "[CLS] " + sentence + " [SEP]"
        tokenized_text = self._tokenizer.tokenize(marked_sentence)
        return tokenized_text

    def get_bert_embeddings(self, sentence):
        """

        :param sentence: input sentence ['str']
        :return: BERT pre-trained hidden states (list of torch tensors) ['List']
        """
        # Predict hidden states features for each layer

        tokenized_text = self.tokenize(sentence)
        indexed_tokens = self._tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            encoded_layers = self._bert_model(tokens_tensor, segments_tensors)

        return encoded_layers[-1][0:12]

    def sentence2vec(self, sentence, layers):
        """

        :param sentence: input sentence ['str']
        :param layers: parameter to decide how word embeddings are obtained ['str]
            1. 'last' : last hidden state used to obtain word embeddings for sentence tokens
            2. 'last_4' : last 4 hidden states used to obtain word embeddings for sentence tokens

        :return: sentence vector [List]
        """
        encoded_layers = self.get_bert_embeddings(sentence)
        
        if layers == 1:
            # using the last layer embeddings
            token_embeddings = encoded_layers[-1]
            # summing the last layer vectors for each token
            sentence_embedding = torch.mean(token_embeddings, 1)
            return sentence_embedding.view(-1).tolist()

        elif layers == 2:
            token_embeddings = []
            tokenized_text = self.tokenize(sentence)

            batch_i = 0
            # For each token in the sentence...
            for token_i in range(len(tokenized_text)):

                # Holds 12 layers of hidden states for each token
                hidden_layers = []

                # For each of the 12 layers...
                for layer_i in range(len(encoded_layers)):
                    # Lookup the vector for `token_i` in `layer_i`
                    vec = encoded_layers[layer_i][batch_i][token_i]

                    hidden_layers.append(list(vec.numpy()))

                token_embeddings.append(hidden_layers)

            # using the last 4 layer embeddings
            token_vecs_sum = []

            # For each token in the sentence...
            for token in token_embeddings:
                # Sum the vectors from the last four layers.
                sum_vec = np.sum(token[-4:], axis=0)

                # Use `sum_vec` to represent `token`.
                token_vecs_sum.append(list(sum_vec))

            # summing the last layer vectors for each token
            sentence_embedding = np.mean(token_vecs_sum, axis=0)
            return sentence_embedding.ravel().tolist()

In [27]:
# Dataset: 3000 chunks * 3 authors

url = 'https://raw.githubusercontent.com/fy164251/text_style_transfer/master/Datasets/raw_text_3000.csv'
df = pd.read_csv(url)

X = df.text.astype('str')
y = df.author.astype('category')

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(y.values)

y = np.asarray(y)
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
encoded = y.reshape(len(y), 1)
y = onehot_encoder.fit_transform(encoded)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
model = Embeddings()

X_text = []
for sentence in tqdm(X):
    X_text.append(model.sentence2vec(sentence, layers=model.LAST_LAYER))

In [28]:
# X_df = pd.DataFrame(X_text)
# X_df.to_csv('./gdrive/My Drive/DL/Style/DistilBert_Embedding_3000.csv')

X_df = pd.read_csv('./gdrive/My Drive/DL/Style/DistilBert_Embedding_3000.csv').set_index('Unnamed: 0')

X_train, X_val, y_train, y_val = train_test_split(X_df, y, stratify=y, random_state=1, test_size=0.2, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1, shuffle=True)

print(X_train.shape, X_val.shape)

(7200, 768) (900, 768)


In [13]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2614,-0.131176,0.266133,0.148921,-0.081992,0.240797,0.114559,0.177920,0.347606,-0.077707,-0.249226,-0.026828,-0.199450,-0.153355,0.141490,-0.139063,0.490254,0.275401,-0.005560,-0.148209,0.194834,0.182864,0.126983,-0.014447,0.492367,0.375321,0.009203,-0.024804,-0.014858,-0.232795,-0.135277,0.193110,0.110863,-0.075256,-0.265667,-0.019461,-0.203811,0.240621,-0.080151,0.055712,0.038758,...,-0.055767,-0.280417,-0.054035,0.328030,-0.125772,-0.201065,0.365234,0.327626,-0.144236,0.004945,-0.197208,-0.018046,0.261606,0.026114,0.139331,-0.357711,-0.121657,0.028659,-0.138672,-0.115437,0.096502,0.072845,0.039624,0.048676,0.275667,-0.195468,-0.262197,-0.287638,0.105576,-0.193612,-0.144565,0.062709,0.076550,-0.242212,-0.015980,-0.177825,0.040596,-0.012199,0.064456,-0.037346
5383,-0.316470,0.154551,-0.044945,-0.193507,0.159468,0.101464,0.108350,0.450681,-0.141811,-0.141473,-0.005227,-0.256182,0.054740,0.135534,-0.370629,0.534947,0.196179,-0.011051,-0.148997,0.012289,0.093987,0.040672,-0.072825,0.373685,0.305296,-0.044486,0.228893,-0.092303,-0.059622,-0.159867,0.473357,0.427233,0.079723,0.078847,-0.323614,-0.110843,0.144783,-0.145783,0.005005,0.103818,...,-0.016835,-0.126305,0.052172,0.114835,-0.064550,-0.179597,0.026677,0.171687,0.091505,-0.021027,0.083243,-0.007528,0.388532,0.065488,0.352679,-0.443432,-0.234592,-0.082353,-0.477019,0.176724,0.014132,0.030622,0.196392,0.064944,0.207005,-0.269276,-0.175285,-0.203082,-0.078734,-0.150577,0.058437,0.180866,-0.137352,-0.297719,0.085819,-0.410921,-0.111298,-0.090707,0.121389,0.118943
2448,0.092707,0.272643,0.176209,-0.011071,0.153302,0.005880,0.030753,0.435551,-0.104705,-0.141612,0.032267,-0.071691,-0.130723,0.386975,-0.050786,0.445956,0.196578,0.104146,0.028415,0.147619,0.151438,0.047032,-0.070511,0.388379,0.296974,0.110106,0.031055,-0.022608,-0.283120,0.126767,0.274742,0.136422,-0.042977,-0.120756,-0.094774,-0.196844,-0.077938,-0.223963,0.032770,0.127072,...,-0.245162,-0.170411,-0.014785,0.122141,-0.168193,-0.117060,0.194875,0.269597,-0.005396,0.039881,-0.070641,0.019881,0.028769,0.117927,0.077930,-0.221662,-0.145010,-0.121209,-0.347139,-0.089856,0.136124,0.160794,0.081143,0.094066,0.478036,-0.168841,-0.209813,-0.216403,-0.003892,-0.019624,0.030146,0.056933,-0.122808,-0.145937,0.055718,-0.297261,-0.016026,-0.006477,0.145422,0.028647
8412,-0.144989,0.249846,-0.032966,-0.074443,0.191077,-0.022496,0.185094,0.596307,-0.191237,-0.145253,-0.021918,-0.042338,0.046627,0.065985,-0.216331,0.465437,0.366972,-0.085402,-0.142934,0.186064,0.039708,0.114470,-0.166818,0.412560,0.276228,0.027942,0.212482,0.051142,-0.154804,-0.165632,0.330933,0.213092,-0.158626,-0.028376,-0.152750,-0.174787,0.176252,-0.161872,0.067942,0.094570,...,-0.047624,-0.291609,-0.034189,0.421426,-0.104581,-0.077029,0.133641,0.313814,-0.171700,-0.083820,-0.114426,-0.016215,0.163013,-0.066032,0.214723,-0.355283,-0.087042,-0.026524,-0.368888,-0.143112,0.103253,0.095139,0.212501,0.010271,0.388163,-0.142768,-0.120668,-0.228226,-0.047740,-0.039385,-0.105743,0.134047,0.118132,-0.287196,0.117638,-0.180990,0.056843,0.011383,-0.013590,0.123433
4448,-0.202350,0.210460,0.008651,0.094393,0.167715,0.033537,-0.074196,0.558049,-0.128097,0.013981,-0.051787,-0.195080,-0.042312,0.189100,-0.341049,0.467573,0.414612,0.037331,-0.096112,0.165886,0.040293,-0.024822,-0.048073,0.480281,0.390848,-0.046351,0.114299,0.008805,-0.113570,-0.146070,0.502898,0.261103,0.096735,-0.035281,-0.313417,-0.172383,0.053957,-0.145919,-0.134239,0.214037,...,0.024570,-0.162921,-0.006297,0.160630,-0.240705,-0.119764,0.144046,0.312836,-0.075514,-0.080052,-0.095569,-0.034356,0.139042,0.055819,0.211669,-0.305341,-0.076088,-0.2084

In [32]:
# Feed-Forward Neural Nets
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Args:
            input_dim (int): the size of the input vectors
            hidden_dim (int): the output size of the first Linear layer
            output_dim (int): the output size of the second Linear layer
        """
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.bn3 = nn.BatchNorm1d(output_dim)

    def forward(self, x):
        """The forward pass of the FFNN
        
        Args:
            x (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim)
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        c = self.fc1(x)
        x = self.bn1(c)
        x = F.relu(x)
        x = F.dropout(x, p=0.5)
        c = torch.cat((x, c), 1)
        x = self.fc2(c)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5)
        c = torch.cat((x, c), 1)
        x = self.fc3(c)
        x = self.bn3(x)
        x = F.relu(x)
        output = F.dropout(x, p=0.5)
     
        return output

batch_size = 32 # number of samples input at once
input_dim = 768
hidden_dim = 128
output_dim = 3

# Initialize model
model = FFNN(input_dim, hidden_dim, output_dim)
print(model)

X = torch.tensor(np.array(X_train))
# y_output = model(X)
# describe(y_output)

FFNN(
  (fc1): Linear(in_features=768, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (bn3): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [18]:
from keras.models import Sequential, Model, load_model
from keras.layers import Embedding, LSTM, Dense, Input, Dropout, GRU, Conv1D, MaxPooling1D, BatchNormalization, Activation, concatenate
from keras.layers import Bidirectional, Flatten, RepeatVector, Permute, Multiply, Lambda, TimeDistributed
from keras import backend as K

from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp36-none-any.whl size=5268 sha256=5909d910deeb66b355b14e57da91dc8e4073a3098e71c7db9244dff45e678ab5
  Stored in directory: /root/.cache/pip/wheels/54/80/22/a638a7d406fd155e507aa33d703e3fa2612b9eb7bb4f4fe667
Successfully built keras-layer-normalization


Using TensorFlow backend.


In [31]:
units = 512
lr = 0.0005
patience = 5


inputs = Input(shape=(768,), dtype='float32')
c = Dense(units)(inputs)
x = BatchNormalization()(c)
x = Dropout(0.5)(x)
x = Activation('relu')(x)

c = concatenate([x, c])

x = Dense(units)(c)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Activation('relu')(x)

c = concatenate([x, c])

x = Dense(3)(c)
x = BatchNormalization()(x)
outputs = Activation('softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adam(lr=lr),
              loss="categorical_crossentropy",
              metrics=["acc"])

model.fit(x=X_train,
          y=y_train,
          validation_data=[X_val, y_val],
          epochs=patience, 
          batch_size=32)

cb = EarlyStopping(monitor='val_loss', 
                   mode='min', 
                   verbose=0, 
                   patience=patience,
                   restore_best_weights=True)

model.compile(optimizer=Adam(lr=lr/3),
              loss="categorical_crossentropy",
              metrics=["acc"])

model.fit(x=X_train, 
          y=y_train,
          validation_data=[X_val, y_val],
          epochs=99, 
          batch_size=32,
          callbacks=[cb])

model.compile(optimizer=Adam(lr=lr/6),
              loss="categorical_crossentropy",
              metrics=["acc"])

model.fit(x=X_train, 
          y=y_train,
          validation_data=[X_val, y_val],
          epochs=99, 
          batch_size=32,
          callbacks=[cb])


print('===Evaluation===')
model.evaluate(X_test, y_test)

Train on 7200 samples, validate on 900 samples
Epoch 1/5
7200/7200 [==============================] - 4s 577us/step - loss: 0.3314 - acc: 0.9568 - val_loss: 0.2225 - val_acc: 0.9878
Epoch 2/5
7200/7200 [==============================] - 2s 285us/step - loss: 0.2467 - acc: 0.9811 - val_loss: 0.1956 - val_acc: 0.9900
Epoch 3/5
7200/7200 [==============================] - 2s 279us/step - loss: 0.2097 - acc: 0.9837 - val_loss: 0.1835 - val_acc: 0.9889
Epoch 4/5
7200/7200 [==============================] - 2s 287us/step - loss: 0.1750 - acc: 0.9868 - val_loss: 0.2150 - val_acc: 0.9667
Epoch 5/5
7200/7200 [==============================] - 2s 282us/step - loss: 0.1494 - acc: 0.9904 - val_loss: 0.1527 - val_acc: 0.9889
Train on 7200 samples, validate on 900 samples
Epoch 1/99
7200/7200 [==============================] - 4s 577us/step - loss: 0.1337 - acc: 0.9903 - val_loss: 0.1468 - val_acc: 0.9922
Epoch 2/99
7200/7200 [==============================] - 2s 283us/step - loss: 0.1261 - acc: 0.9

[0.01402373373715414, 0.9955555555555555]